<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Neural-Nets-Are-Not-Black-Boxes" data-toc-modified-id="Neural-Nets-Are-Not-Black-Boxes-1">Neural Nets Are Not Black Boxes</a></span></li><li><span><a href="#Objective" data-toc-modified-id="Objective-2">Objective</a></span></li><li><span><a href="#Back-propagation-is-the-Chain-Rule" data-toc-modified-id="Back-propagation-is-the-Chain-Rule-3">Back-propagation is the Chain Rule</a></span></li><li><span><a href="#Binary-Cross-Entropy" data-toc-modified-id="Binary-Cross-Entropy-4">Binary Cross Entropy</a></span></li><li><span><a href="#Activations" data-toc-modified-id="Activations-5">Activations</a></span></li><li><span><a href="#Linear-Layer" data-toc-modified-id="Linear-Layer-6">Linear Layer</a></span></li><li><span><a href="#Putting-It-All-Together" data-toc-modified-id="Putting-It-All-Together-7">Putting It All Together</a></span></li><li><span><a href="#Our-Evaluation-Metric" data-toc-modified-id="Our-Evaluation-Metric-8">Our Evaluation Metric</a></span></li><li><span><a href="#Trainer" data-toc-modified-id="Trainer-9">Trainer</a></span></li><li><span><a href="#Pre-process-Data" data-toc-modified-id="Pre-process-Data-10">Pre-process Data</a></span></li><li><span><a href="#Datasets-&amp;-DataLoaders" data-toc-modified-id="Datasets-&amp;-DataLoaders-11">Datasets &amp; DataLoaders</a></span></li><li><span><a href="#Train" data-toc-modified-id="Train-12">Train</a></span></li><li><span><a href="#Comparison-with-sklearn" data-toc-modified-id="Comparison-with-sklearn-13">Comparison with sklearn</a></span></li><li><span><a href="#Adding-a-Second-Layer" data-toc-modified-id="Adding-a-Second-Layer-14">Adding a Second Layer</a></span></li></ul></div>

# Neural Nets Are Not Black Boxes 

If you think neural nets are black boxes, you're certainly not alone. While they may not be as interpretable as something like a random forest (at least not yet), neural nets are in no way black boxes. In this post we'll break down this misconception piece by piece as we build our own network from scratch, starting with logistic regression.    

This post is very much inspired by [this fantastic post](https://sgugger.github.io/a-simple-neural-net-in-numpy.html#a-simple-neural-net-in-numpy) by Sylvain Gugger. We won't pretend to add to or improve upon Sylvain's post; we just want to explain things in our own way to help us understand things a little bit more clearly. This will be the first of a series of posts in which we'll write our own DNN, CNN, and RNN from scratch. You can find the source code for all of these posts at our [tinytorch repo](https://github.com).

In [1]:
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [2]:
# Set seed for reproducibility
seed = 9
np.random.seed(seed)

# Objective 

Our goal is to construct a binary logistic classifier as a neural network. The network will consist of a single linear layer followed by a sigmoid activation with binary cross entropy as the loss function. We'll begin by deriving the back-prop equations for our particular scenario and in the process we'll realize that what we've done generalizes immediately to networks with any number of layers and arbitrary activations. In other words, we'll have developed a framework that can model any feedforward network--all by starting with ordinary logistic regression. 

Actually, this isn't all that surprising when you think about it, since logistic regression is a linear layer followed by sigmoid and feedforward networks are just a bunch of linear layers stacked together with non-linear activations in between.

# Back-propagation is the Chain Rule

Back-propagation is nothing more than the chain rule. We can view our logistic network as the composition of three functions

$$x \to \text{BCE} \circ \text{Sigmoid} \circ \text{Linear}(x)$$

While the loss function is not usually viewed as a layer of the network, treating it as the final layer makes computing the gradients easier. Let's denote the output of the $i$-th layer by $x_i$ so that

\begin{align}
    x_1 &= \text{Linear}(x)     \\
    x_2 &= \text{Sigmoid}(x_1)  \\
    x_3 &= \text{BCE}(x_2)
\end{align}

The first gradient we have to compute is the gradient of $\text{BCE}$ with respect to the activations $x_2$

$$\frac{\partial \text{BCE}}{\partial x_2} = \frac{\partial \text{BCE}}{\partial x_2}(x_2) $$

Next we have to compute the gradient with respect to the linear outputs $x_1$. The chain rule tells us

$$\frac{\partial \text{BCE}}{\partial x_1} = \frac{\partial \text{BCE}}{\partial x_2} \times \frac{\partial \text{Sigmoid}}{\partial x_1}(x_1)$$

Last, we'll need to compute the gradient with respect to the original inputs $x$

$$\frac{\partial \text{BCE}}{\partial x} = \frac{\partial \text{BCE}}{\partial x_1} \times \frac{\partial \text{Linear}}{\partial x_1}(x)$$

See the pattern? The first gradient we computed--the gradient with respect to the network's final activations--is used to compute the following gradient--the gradient with respect to the linear outputs--which are in turn used to compute the gradient with respect to the original inputs. To compute the gradients of any network, we simply start at the last layer and successively pass the gradients backwards to the preceding layer until we arrive at our original inputs. That's why it's called back-propagation and not "just the chain rule". It really is helpful to picture passing the gradients backwards through the network like a baton.

We'll compute each of these gradients in turn, starting with the final layer and working our way backwards to the original inputs.

__Note:__ So far we've been treating the input $x$ as a single variable, but most of the time $x$ will have more than one dimension. Computing the gradients in the multi-variate case is more or less the same (although it involves something called the Jacobian, but we'll pretend we didn't hear that).

--------

# Binary Cross Entropy

Binary cross entropy penalizes predictions by the logarithm of their confidence. Given labels $y$ which are either zero or one and probabilities $\hat{y}$ for the positive class, we add $-\ln(\text{P}(y=1))$ to the loss whenever $y = 1$ and $-\ln(\text{P}(y=0))$ whenever $y = 0$. In other words,

$$\text{BCE}(\hat{y}, y) = -[y \ln(\hat{y}) + (1 - y)\ln(1 - \hat{y})]$$

After simplifying, you'll find its derivative is

$$\frac{\partial \text{BCE}}{\partial \hat{y}} = \frac{\hat{y} - y}{\hat{y}(1 - \hat{y})}$$

To avoid potential division-by-zero errors, we'll clip the output probabilities $\hat{y}$ so that they're not too close to either zero or one.

In [57]:
class BinaryCrossEntropy:
    """Container for the forward and backward pass of BCE."""
    
    def forward(self, y_hat, y):
        """Return binary cross entropy given predictions and targets."""
        self.y_hat, self.y = y_hat.clip(min=1e-8, max=1-1e-8), y
        return -np.where(y==1, np.log(self.y_hat), np.log(1 - self.y_hat))
    
    def backward(self):
        """Backpropagate the gradient with respect to predictions."""
        return (self.y_hat - self.y) / (self.y_hat * (1 - self.y_hat))

-----

# Activations

The easiest components to handle are the activation functions. Our activation is sigmoid, which you'll often see defined as one of 

$$\sigma(x) = \frac{1}{1 + \text{exp}(-x)} \quad \text{or} \quad \frac{\text{exp}(x)}{1 + \text{exp}(x)}$$

It turns out that we need both versions to implement a numerically stable version of sigmoid. Why? Notice how when $x$ is very negative, $\text{exp}(-x)$ is very large, and when $x$ is very positive, $\text{exp}(x)$ is very large--in both cases too large to store in memory. The easy fix is to use the first version when $x > 0$ and the second when $x < 0$.

After simplifying, you'll find the derivative of sigmoid is

$$\frac{\partial \sigma}{\partial x} = \sigma(x)(1 - \sigma(x))$$

Notice something interesting? Since the probabilities $\hat{y}$ are the outputs $\sigma(x)$ of sigmoid, this is exactly the denominator of the BCE gradient we computed above, meaning the two terms will cancel when we compute the gradient of BCE with respect to the outputs $x_1$ of our network's final (and only) linear layer

$$\frac{\partial \text{BCE}}{\partial x_1} = \hat{y} - y $$

In other words, the gradient of the loss with respect to the network's final linear outputs is just the difference between the probabilities $\hat{y}$ and the labels $y$. The further apart they are (i.e. the worse our predictions are), the larger the gradient and the larger the update to the last linear layer's weights in the SGD step (remember, the chain rule tells us that the gradient with respect to the final linear outputs appear as a factor in the gradient with respect to the last linear layer's weights themselves). 

This is terrific because it means the weights of our network will change gradually as we train and won't spike or drop suddenly, which would be the case if the gradients were a quadratic or higher-order function of the prediction errors. It also demonstrates nicely how a network adjusts its weights based on the error of its predictions.

In [4]:
class Sigmoid:
    """Container for the forward and backward pass of sigmoid."""
    
    def forward(self, x):
        """Pass a mini-batch through a sigmoid layer."""
        self.y_hat = np.where(x > 0, 1 / (1 + np.exp(-x)), np.exp(x) / (1 + np.exp(x)))
        return self.y_hat
        
    def backward(self, grad):
        """Backpropagate the gradient given the preceding gradient."""
        return self.y_hat * (1 - self.y_hat) * grad

----------

# Linear Layer

The last and most difficult component we need to implement is the linear layer, which contains weights and biases. Denoting the output of this layer by $z$, we have

$$z = xw + b$$

If $x$ is a mini-batch of shape $(bs, n_{inp})$ then $w$ has shape $(n_{inp}, 1)$ and $b$ has shape $(1,)$, with addition being done via broadcasting. To make things easier, for the moment let's just imagine we have a batch size of one:

$$x = [x_1, \dots, x_{n_{inp}}]$$ 

There are two gradients to compute this time around, one with respect to the weights and another with respect to the bias. To make life easier, let's everything out in terms of coordinates:

$$z_i = \sum_{k=1}^{n_{inp}} x_k w_{ki} + b$$

Then we get

$$\frac{\partial \text{BCE}}{\partial w_{ki}} = \frac{\partial \text{BCE}}{\partial z_i} \times \frac{\partial z_i}
{\partial w_{ki}} = \frac{\partial \text{BCE}}{\partial z_i} \times x_k$$

and

$$\frac{\partial \text{BCE}}{\partial b} = \frac{\partial \text{BCE}}{\partial z_i} \times \frac{\partial z_i}{\partial b} = \frac{\partial \text{BCE}}{\partial z_i}$$

Remember, we'll already have the gradient with respect to the output $z$ of the linear layer stored beforehand (say in a variable called $\text{grad}$) when it's time to compute the gradient with respect to the weights and biases. Notice something really nice? $\text{grad}$ is exactly the gradient with respect to $b$, leaving only the gradient with respect to the weights to compute. The main thing we need to do is figure out how to write this as a matrix product.

Whenever I have a hard time doing something like this, I just focus on getting the shapes right:

\begin{align}
  &\bullet x \text{ has shape } (bs, n_{inp}) \\
  &\bullet \text{grad has shape } (bs, 1) \\
  &\bullet \text{grad}_w \text{ has shape } (n_{inp}, 1)
\end{align}

The only way we can multiply $x$ and $\text{grad}$ and get something of shape $(n_{inp}, 1)$ is to resize $x$ to have shape $(bs, n_{inp}, 1)$ and $\text{grad}$ to have shape $(bs, 1, 1)$ so that ordinary matrix multiplication over the last two dimensions gives the shape $(n_{inp}, 1)$.

Note that were there another linear layer we would also need to compute the gradient of the loss with respect to the inputs $x$ so that we could keep back-propagating the gradients. This isn't anymore complicated than what we've done so far and doing so will allow us to build networks with any number of layers, so let's go ahead and do it! Since $x_k$ appears in each of the activations $z_i$, the gradient will respect to $x_k$ will involve summing up all of the intermediate gradients with respect to $z_i$:

$$ \frac{\partial \text{BCE}}{x_k} = \sum_{i=1}^{n_{inp}} \frac{\partial \text{BCE}}{\partial z_i} \times \frac{\partial z_i}{\partial x_k} = \sum_{i=1}^{n_{inp}} \frac{\partial \text{BCE}}{\partial z_i} w_{ki}$$

Re-writing this as a matrix product, we get

$$ \frac{\partial \text{BCE}}{x_k} = \text{grad} \times W^t$$

where $W$ is the weight matrix. Let's do a sanity check on the dimensions involved to make sure nothing has gone horribly wrong. Since $\text{grad}$ has shape $(bs, 1)$ and $W$ has shape $(n_{inp}, 1)$, $\text{grad} \times W^t$ has shape $(bs, n_{inp})$, which is exactly the shape of $x$--just as it should be.

In [5]:
class Linear:
    """Container for the forward and backward pass of a linear layer."""
    
    def __init__(self, n_inp, n_out):
        k = 1 / np.sqrt(n_inp)
        self.weights = np.random.uniform(-k, k, (n_inp, n_out))
        self.bias = np.zeros(n_out)
        
    def forward(self, x):
        """Pass a mini-batch through a linear layer."""
        self.x = x
        return x @ self.weights + self.bias
    
    def backward(self, grad):
        """Compute the gradients of the weights and biases given previous gradient."""
        self.grad_w = (self.x[:,:,None] @ grad[:,None,:]).mean(axis=0)
        self.grad_b = grad.mean(axis=0)
        return grad @ self.weights.T

---------

# Putting It All Together

Wohoo! It's finally time to string together all the work we've done so far into a single network.

In [6]:
class Sequential:
    """Container for a feedforward neural net."""
    
    def __init__(self, layers, criterion, metric):
        """Initialise layers, loss criterion, and evaluation metric."""
        self.layers = layers
        self.criterion = criterion
        self.metric = metric
        
    def forward(self, x):
        """Pass a mini-batch through the network."""
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward(self):
        """Backpropagate gradients to the start of the network."""
        grad = self.criterion.backward()
        for layer in self.layers[::-1]:
            grad = layer.backward(grad)

In [7]:
class SGD:
    """Container for updating a model's weights via SGD."""
    
    def __init__(self, model, lr):
        self.model = model
        self.lr = lr
                  
    def step(self):
        """Update the weights and biases of all linear layers."""
        for layer in self.model.layers:
            if isinstance(layer, Linear):
                layer.weights -= self.lr * layer.grad_w
                layer.bias -= self.lr * layer.grad_b

----------

# Our Evaluation Metric

For simplicity, we'll just consider accuracy for the time being

In [8]:
def accuracy(y, y_hat):
    """Compute accuracy given soft binary predictions."""
    y_pred = y_hat > 0.5
    return (y_pred == y).mean()

---------

# Trainer

To make life easier, let's wrap all of the functionality we need to train a network into a single class.

In [9]:
class Trainer:
    """Container for training a feedforward neural net."""
    
    def __init__(self, model, optimizer, train_dl, val_dl):
        self.model = model
        self.optimizer = optimizer
        self.train_dl = train_dl
        self.val_dl = val_dl
        
    def _train(self):
        """Train for a single epoch and return the loss."""
        loss, n = 0, 0
        for x, y in self.train_dl:
            y_hat = self.model.forward(x)
            batch_loss = self.model.criterion.forward(y, y_hat).sum()
            self.model.backward()
            self.optimizer.step()
            loss += batch_loss
            n += len(y)
        return loss / n
            
    def train(self, n_epochs, log_level=1):
        """Train for multiple epochs."""
        for epoch in range(n_epochs):
            loss = self._train()
            self.optimizer.step()
            val_loss, val_metric = self.evaluate(self.val_dl)
            if (epoch + 1) % log_level == 0:
                print(f"{epoch= :2d} | {loss= :.3f} | {val_loss= :.3f} | {val_metric= :.3f}")
    
    def evaluate(self, dl):
        """Return loss and metric on validation or test set."""
        loss, n, metric = 0, 0, 0
        for x, y in dl:
            y_hat = self.model.forward(x)
            batch_loss = self.model.criterion.forward(y, y_hat).sum()
            batch_metric = self.model.metric(y, y_hat)
            metric += len(y) * batch_metric
            loss += batch_loss
            n += len(y)
        return loss / n, metric / n

---------

# Pre-process Data

We'll use sklearn's [breast cancer dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer) for our binary classification task.

In [10]:
# Load data
X, y = load_breast_cancer(return_X_y=True)
X.shape, y.shape

((569, 30), (569,))

In [11]:
# Train-test-split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=seed)
X_train.shape, X_val.shape

((455, 30), (114, 30))

In [12]:
# Normalize
mu, sigma = X_train.mean(), X_train.std()
X_train = (X_train - mu) / sigma
X_val = (X_val - mu) / sigma

---------

# Datasets & DataLoaders

In [13]:
class Dataset:
    """Container for returning inputs and targets."""
    
    def __init__(self, X, y):
        self.X = X
        self.y = y.reshape(-1, 1)
        
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
    def __setitem__(self, idx, val):
        self.X[idx], self.y[idx] = val
                
    def __len__(self):
        return len(self.y)

In [14]:
class DataLoader:
    """Container for returning a mini-batch of inputs and targets."""
    
    def __init__(self, ds, batch_size, shuffle=False):
        self.ds = ds
        self.batch_size = batch_size
        self.shuffle = shuffle
        
    def shuffle_data(self):
        """Shuffle inputs and targets."""
        idxs = np.random.permutation(len(self.ds))
        self.ds = Dataset(*self.ds[idxs])
        
    def __iter__(self):
        """Yield a mini-batch of inputs and targets."""
        if self.shuffle: self.shuffle_data()
        n_batches = len(self.ds) // self.batch_size
        for i in range(n_batches):
            yield self.ds[i * self.batch_size: (i + 1) * self.batch_size]

In [15]:
# Load training and validation data
train_ds = Dataset(X_train, y_train)
val_ds = Dataset(X_val, y_val)

batch_size = 64
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=len(X_val), shuffle=False)

--------

# Train

Now we're ready to put our model to the test.

In [44]:
# Input and final output dims
n_inp = X_train.shape[1]

# Initialise layers and criterion
metric = accuracy
criterion = BinaryCrossEntropy()
layers = [Linear(n_inp, 1), Sigmoid()]
model = Sequential(layers, criterion, metric)

# Initialise optimizer and trainer
optimizer = SGD(model, lr=0.1)
trainer = Trainer(model, optimizer, train_dl, val_dl)

In [47]:
trainer.train(10)

epoch=  0 | loss= 0.619 | val_loss= 0.577 | val_metric= 0.798
epoch=  1 | loss= 0.559 | val_loss= 0.515 | val_metric= 0.895
epoch=  2 | loss= 0.514 | val_loss= 0.472 | val_metric= 0.895
epoch=  3 | loss= 0.478 | val_loss= 0.440 | val_metric= 0.904
epoch=  4 | loss= 0.449 | val_loss= 0.412 | val_metric= 0.895
epoch=  5 | loss= 0.421 | val_loss= 0.390 | val_metric= 0.895
epoch=  6 | loss= 0.402 | val_loss= 0.374 | val_metric= 0.904
epoch=  7 | loss= 0.388 | val_loss= 0.362 | val_metric= 0.895
epoch=  8 | loss= 0.370 | val_loss= 0.343 | val_metric= 0.904
epoch=  9 | loss= 0.359 | val_loss= 0.332 | val_metric= 0.895


-------------

# Comparison with sklearn

Let's compare how our logistic network stacks up against sklearn's own logistic regression model.

In [18]:
# We're close!
sklearn_model = LogisticRegression(random_state=seed)
sklearn_model.fit(X_train, y_train)
sklearn_model.score(X_val, y_val)

0.9122807017543859

----------------

# Adding a Second Layer

Let's add another linear layer to our network, followed, of course, by an activation.

In [50]:
class ReLU:
    """Container for the forward and backward pass of ReLU."""
    
    def forward(self, x):
        """Pass a mini-batch through ReLU."""
        self.x = x
        return np.where(x > 0, x, 0)
    
    def backward(self, grad):
        """Return the gradient where x is positive, otherwise zero."""
        return np.where(self.x > 0, grad, 0)

In [51]:
# Input and final output dims
n_inp = X_train.shape[1]

# Initialise layers and criterion
metric = accuracy
criterion = BinaryCrossEntropy()
layers = [Linear(n_inp, 20), ReLU(), Linear(20, 1), Sigmoid()]
model = Sequential(layers, criterion, accuracy)

# Initialise optimizer and trainer
optimizer = SGD(model, lr=0.10)
trainer = Trainer(model, optimizer, train_dl, val_dl)

In [54]:
trainer.train(12)

epoch=  0 | loss= 0.683 | val_loss= 0.654 | val_metric= 0.868
epoch=  1 | loss= 0.639 | val_loss= 0.611 | val_metric= 0.851
epoch=  2 | loss= 0.597 | val_loss= 0.562 | val_metric= 0.912
epoch=  3 | loss= 0.554 | val_loss= 0.515 | val_metric= 0.895
epoch=  4 | loss= 0.512 | val_loss= 0.473 | val_metric= 0.868
epoch=  5 | loss= 0.468 | val_loss= 0.422 | val_metric= 0.904
epoch=  6 | loss= 0.425 | val_loss= 0.382 | val_metric= 0.895
epoch=  7 | loss= 0.387 | val_loss= 0.367 | val_metric= 0.904
epoch=  8 | loss= 0.364 | val_loss= 0.330 | val_metric= 0.912
epoch=  9 | loss= 0.335 | val_loss= 0.303 | val_metric= 0.886
epoch= 10 | loss= 0.311 | val_loss= 0.280 | val_metric= 0.904
epoch= 11 | loss= 0.292 | val_loss= 0.267 | val_metric= 0.904
